In [3]:
from os import listdir
from os.path import join
import regex as re
import numpy as np
from random import randint


def splitContentBySections(content: str):
    sections = [{'content': '# ' + section} for section in filter(None, re.split("\n##(?=[^#])", content))]
    
    for section in sections:
        section['labels'] = labelSection(section['content'])
        
    sections[0]['labels'].append('first')
        
    return sections

def findRegexHeadings(header):
    return f'(#+\s*(?:\d*|[ivx]*)\.?\s*{header})'

def getGenericHeadingCheckerFunction(*args):    
    return (lambda s: np.array([bool(re.findall(findRegexHeadings(header),s)) for header in args]).any())
    

def labelSection(content: str):    
    mappings = {
                        'reference': getGenericHeadingCheckerFunction('references', 'citations'),
                        'method': getGenericHeadingCheckerFunction('methodology', 'method', 'approach', 'experiment'),
                        'abstract': getGenericHeadingCheckerFunction('abstract'),
                        'appendix': getGenericHeadingCheckerFunction('appendix'),
                        'background': getGenericHeadingCheckerFunction('introduction', 'related work', 'background'),
                        'conclusion': getGenericHeadingCheckerFunction('conclusion', 'discussion'),
                        'results': getGenericHeadingCheckerFunction('results')
    }
    
    labels = [label for label, func in mappings.items() if func(content)]
    return labels



<>:19: SyntaxWarning: invalid escape sequence '\s'
<>:19: SyntaxWarning: invalid escape sequence '\s'
/var/folders/6j/9qz49mv9637bsngvd04d4pf00000gp/T/ipykernel_15017/3414380311.py:19: SyntaxWarning: invalid escape sequence '\s'
  return f'(#+\s*(?:\d*|[ivx]*)\.?\s*{header})'


In [6]:
markdown_path = '../data/Markdown'
all_markdowns = listdir(markdown_path)

num = randint(1, len(all_markdowns)-1)
print(num)
sample = all_markdowns[num]
print(sample)
with open(join(markdown_path, sample), 'r') as f:
    content = f.read().lower()
sections = splitContentBySections(content)
sections


1669
de519a1976d0f5005ffac09f2560b1b61b37603c.mmd


[{'content': "# # chatlaw: open-source legal large language model with integrated external knowledge bases\n\n jiaxi cui\n\npeking university\n\njiaxicui@chatlaw.cloud\n\nequal contribution.\n\nzongjian li\n\npeking university\n\nchestnutlzj@chatlaw.cloud\n\nyang yan\n\npeking university\n\nyyang@stu.pku.edu.cn\n\n&bohua chen\n\npeking university\n\nbohua@chatlaw.cloud\n\n&li yuan\n\npeking university\n\nyuanli-ece@pku.edu.cn\n\ncorresponding author\n\n###### abstract\n\nlarge language models (llms) have shown the potential to revolutionize natural language processing tasks in various domains, sparking great interest in vertical-specific large models. however, unlike proprietary models such as bloomberggpt and fingpt, which have leveraged their unique data accumulations to make strides in the finance domain, there hasn't not many similar large language models in the chinese legal domain to facilitate its digital transformation.\n\nin this paper, we propose an open-source legal large la

In [ ]:
markdown_sections = []
percentages = []
for markdown in all_markdowns:
    with open(join(markdown_path, markdown), 'r') as f:
        content = f.read().lower()
    
    sections = splitContentBySections(content)
    has_method = len([sec for sec in sections if 'method' in sec['labels']]) > 0
    markdown_sections.append((sections, has_method))
    
    if has_method:
        method_length = np.array([len(section['content']) for section in sections if 'method' in section['labels']]).sum()
        all_length = np.array([len(section['content']) for section in sections]).sum()
        percentages.append(method_length/all_length)
        
print(f"Average content percent as method: {np.array(percentages).mean()}")


In [ ]:
len([sections for sections, has_method in markdown_sections if not has_method])/len(all_markdowns)

i += 1
[sections for sections, has_method in markdown_sections if not has_method][i]